In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_TA-14-19854C-400-008.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_PT-14-21998AB-400-045.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_PT-14-29315EF-400-006.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_F-14-23060CD-400-009.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_A-14-22549G-400-028.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_A-14-22549CD-400-029.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-019.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_F-14-21998EF-400-004.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_PT-14-29315EF-400-008.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_A-14-22549AB-400-027.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_F-14-25197-400-056.png
/kaggle/input/breakhis-400x/BreaKHis 400X/test/benign/SOB_B_TA-14

In [2]:
import numpy as np
import pandas as pd
import cv2
import os
from zipfile import ZipFile
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.utils import img_to_array
from keras.utils import np_utils
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense, Dropout
from keras.models import Sequential


In [3]:
dataset = []
labels = []


In [4]:
def prepare_benign_images(path):
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    try:
        img_res = cv2.resize(img,(256,256))
        img_array = img_to_array(img_res)
        img_array = img_array/255.0
        dataset.append(img_array)
        labels.append('0')
    except:
        print("error")

In [5]:
def prepare_malignant_images(path):
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    try:
        img_res = cv2.resize(img,(256,256))
        img_array = img_to_array(img_res)
        img_array = img_array/255.0
        dataset.append(img_array)
        labels.append('1')
    except:
        print("error")

In [6]:
from os import listdir
 
## pre processing train benign images
folder_dir = "/kaggle/input/breakhis-400x/BreaKHis 400X/train/benign"
for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".png")):
        path = os.path.join('/kaggle/input/breakhis-400x/BreaKHis 400X/train/benign',images)
        prepare_benign_images(path)

In [7]:
## pre processing train malignant images
folder_dir = "/kaggle/input/breakhis-400x/BreaKHis 400X/train/malignant"
for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".png")):
        path = os.path.join('/kaggle/input/breakhis-400x/BreaKHis 400X/train/malignant',images)
        prepare_malignant_images(path)

In [8]:
len(dataset)

1148

In [25]:
images = np.array(dataset)
label_arr = np.array(labels)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(images,label_arr,stratify=label_arr,test_size=0.10,random_state=44)
y_train=np_utils.to_categorical(y_train,num_classes=2)
y_test=np_utils.to_categorical(y_test,num_classes=2)


In [26]:
input_shape = (128,256,256,3)
n_classes = 2

resnet_model = tf.keras.applications.ResNet50(
input_shape= (256,256,3),
include_top=False,
weights = 'imagenet'
)

resnet_model.trainable = False

resnet_model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [27]:
resnettrain = tf.keras.Sequential([
    resnet_model,
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

resnettrain.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 2048)       8192      
 ormalization)                                                   
                                                                 
 flatten_4 (Flatten)         (None, 131072)            0         
                                                                 
 dense_7 (Dense)             (None, 16)                2097168   
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 8)                 136       
                                                        

In [29]:
resnettrain.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)
history = resnettrain.fit(
    x_train,
    y_train,
    batch_size=128,
    verbose=1,
    epochs=50
    
)

Epoch 1/50
9/9 [==============================] - 9s 442ms/step - loss: 0.8983 - accuracy: 0.5528
Epoch 2/50
9/9 [==============================] - 4s 451ms/step - loss: 0.8661 - accuracy: 0.5218
Epoch 3/50
9/9 [==============================] - 4s 448ms/step - loss: 0.8223 - accuracy: 0.5063
Epoch 4/50
9/9 [==============================] - 4s 449ms/step - loss: 0.8261 - accuracy: 0.5373
Epoch 5/50
9/9 [==============================] - 4s 448ms/step - loss: 0.8088 - accuracy: 0.5479
Epoch 6/50
9/9 [==============================] - 4s 446ms/step - loss: 0.7872 - accuracy: 0.5402
Epoch 7/50
9/9 [==============================] - 4s 438ms/step - loss: 0.8152 - accuracy: 0.5353
Epoch 8/50
9/9 [==============================] - 4s 443ms/step - loss: 0.8161 - accuracy: 0.5392
Epoch 9/50
9/9 [==============================] - 4s 433ms/step - loss: 0.7392 - accuracy: 0.5915
Epoch 10/50
9/9 [==============================] - 4s 433ms/step - loss: 0.7194 - accuracy: 0.5992
Epoch 11/50
9/9 [==

In [30]:
pred = resnettrain.predict(x_test)

resnettrain.evaluate(x_test,y_test)

4/4 [==============================] - 2s 105ms/step - loss: 0.6285 - accuracy: 0.6783


[0.6285434365272522, 0.678260862827301]

NameError: name 'accuracy_score' is not defined

NameError: name 'conMat' is not defined